In [1]:
import json
import pennylane as qml
import pennylane.numpy as np



In [2]:
# Put any helper functions here that you want to make #


def encode_hermitian(A, wires):
    """
    Encodes a hermitian matrix A as a unitary U = e^{iA}.

    Args
        - A (numpy.tensor): a 2x2 matrix
        - b (numpy.tensor): a length-2 vector

    Returns
        - (qml.Operation): a unitary operation U = e^{iA}
    """
    return qml.exp(qml.Hermitian(A, wires=wires), coeff=1j)




In [34]:
def control_R(qpe_wires,ancilla_wires):
    count = 0
    for i in qpe_wires:
        #ang = 2*np.arcsin(2**(-count))
        ang = 2*np.arcsin(2**(-count))
        #ang = np.pi*2**(count-len(qpe_wires))
        qml.ctrl(qml.RY(ang,ancilla_wires),i)
        count += 1

In [ ]:
def mint_to_lime(A, b):
    """
    Calculates the optimal mint and lime proportions in the Mojito HHLime twist.

    Args
        - A (numpy.tensor): a 2x2 matrix
        - b (numpy.tensor): a length-2 vector

    Returns
        - x (numpy.tensor): the solution to Ax = b
        (int): the number of operations in your HHL circuit.
    """
    b_qubits = 1
    b_wires = [0]

    qpe_qubits = 10
    qpe_wires = list(range(b_qubits, b_qubits + qpe_qubits))

    ancilla_qubits = 1
    ancilla_wires = list(
        range(b_qubits + qpe_qubits, ancilla_qubits + b_qubits + qpe_qubits)
    )

    all_wires = b_wires + qpe_wires + ancilla_wires
    dev = qml.device("default.qubit", wires=all_wires)
    @qml.qnode(dev)
    def HHL(A, b):
        """
        Implements the HHL algorithm.
        Args
            - A (numpy.tensor): a 2x2 matrix
            - b (numpy.tensor): a length-2 vector

        Returns
            - (numpy.tensor):
                The probability distribution for the vector x, which is the
                solution to Ax = b.
        """


        # Put your code here #
        # state preparation
        qml.QubitStateVector(b, b_wires)
        # prepare unitary
        U = encode_hermitian(A, b_wires)
        # do QPE
        qml.QuantumPhaseEstimation(U, estimation_wires=qpe_wires)
        # do CRY, from register to ancilla
        control_R(qpe_wires,ancilla_wires)
        # do IQPE
        qml.adjoint(qml.QuantumPhaseEstimation)(U, estimation_wires=qpe_wires)

        if np.allclose(qml.probs(wires=[1]),np.array([0.,1.]), rtol=1e-2):
            return qml.probs(wires=b_wires)
        else:
            dev = qml.device("default.qubit", wires=all_wires)
            return qml.QNode(HHL(A, b), dev)
    # we return probs, but we need the state itself (it will be real-valued)
    x = np.sqrt(HHL(A, b))
    #drawer = qml.draw(HHL)
    #print(drawer(A, b))
    return x, len(HHL.tape._ops)




In [ ]:
# These functions are responsible for testing the solution.
test = [[[1, -0.333333], [-0.333333, 1]], [0.48063554, 0.87692045]]

In [37]:
mint_to_lime(test[0],test[1])

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [48]:
def test_op():
    qml.PauliX(0)
    qml.PauliY(0)

In [90]:
dev1 = qml.device("default.qubit", wires=1, shots=1000)
@qml.qnode(dev1)
def getState():
    qml.PauliX(0)
    return qml.counts()

In [115]:
getState().get('0')
